## Experimento de Shannon

O notebook a seguir é uma réplica do experimento de Claude Shannon (usando computação) destacado no seu artigo A Mathematical Theory of Communication de 1948.

- O experimento consiste em gerar sequências de texto com graus variados de sofisticação estatística para ilustrar como diferentes ordens de n-gramas (ideia de Markov) alteram a similaridade com o inglês real.


In [40]:
# Dependências
import re
import random
from datasets import load_dataset
from typing import Dict, List, Tuple, Optional
from collections import Counter, defaultdict


seed = 42
random.seed(seed)

## Carregando dados

O conjunto de dados escolhido foi um coleção de livros em inglês, onde iremos normalizar o texto para facilitar o processo estatístico de contabilizar os grams.

In [22]:
config_hf = "books"
split = "train"

### Normalização dos dados

In [23]:
def manter_letras_e_espacos(texto: str) -> str:
    if texto is None:
        return ""
    s = str(texto).lower()
    return re.sub(r"[^a-z ]+", "", s)

def adicionar_coluna_normalizada(exemplo):
    exemplo["texto_normalizado"] = manter_letras_e_espacos(exemplo.get("text", ""))
    return exemplo


In [24]:
dataset = load_dataset("ubaada/booksum-complete-cleaned", config_hf, split=split)
dataset = dataset.map(adicionar_coluna_normalizada)

In [ ]:
def juntar_corpus(ds, coluna: str, max_docs: Optional[int] = 200) -> str:
    n = len(ds) if max_docs is None else min(max_docs, len(ds))
    return " ".join(ds[i][coluna] for i in range(n) if ds[i][coluna])

In [37]:
alfabeto = list("abcdefghijklmnopqrstuvwxyz ")
corpus = juntar_corpus(dataset, "texto_normalizado", max_docs=200)
print(f"Tamanho do corpus: {len(corpus)}")

Tamanho do corpus: 69863990


## Experiemento

In [29]:
alfabeto = list("abcdefghijklmnopqrstuvwxyz ")
tamanho = 200

### Amostras aleatórias

In [35]:
def amostrar_caracteres_equiprovaveis(alfabeto: List[str], tamanho: int, seed: Optional[int] = None) -> str:
    rng = random.Random(seed) if seed is not None else random
    return "".join(rng.choice(alfabeto) for _ in range(tamanho))

In [30]:
amostra = amostrar_caracteres_equiprovaveis(alfabeto, tamanho=tamanho, seed=seed)
print(amostra)

udaxihhexdvxrcsnbacghqtargwuwrnhosizayzfwnkiegykdcmdlltizbxordmcrj utlsgwcbvhyjchdmiou lfllgviwvuctufrxhfomiuwrhvk yybh bzkmicgswkgupmuoeiehxrrixsnsmlheqpcybdeufzvntcmmtoqiravxdvryiyukdjnfoaxxiqyfqduj


Aqui é possível perceber que o aleatório é realmente aleatório rsrs, mas não parece seguir nenhum "padrão" da linguagem

### Trabalhando com cadeias de markov

In [41]:
def treinar_unigramas(texto: str) -> Counter:
    return Counter(ch for ch in texto)

def treinar_ngramas(texto: str, n: int) -> Tuple[Dict[str, Counter], Counter]:
    assert n >= 1
    unigr = treinar_unigramas(texto)
    if n == 1:
        return {}, unigr
    modelo = defaultdict(Counter)
    janela = n - 1
    for i in range(len(texto) - janela):
        contexto = texto[i:i+janela]
        prox = texto[i+janela]
        modelo[contexto][prox] += 1
    return dict(modelo), unigr

def amostrar_com_pesos(contagens: Counter, rng: random.Random) -> str:
    simbolos = list(contagens.keys())
    pesos = list(contagens.values())
    return rng.choices(simbolos, weights=pesos, k=1)[0]

In [42]:
def escolher_contexto_inicial(modelo: Dict[str, Counter], janela: int, rng: random.Random) -> str:
    if modelo:
        return rng.choice(list(modelo.keys()))
    return " " * janela

def amostrar_markov(modelo: Dict[str, Counter],
                    unigr: Counter,
                    n: int,
                    tamanho: int,
                    alfabeto: List[str],
                    seed: Optional[int] = None) -> str:
    rng = random.Random(seed) if seed is not None else random
    if n == 1:
        return "".join(rng.choices(list(unigr.keys()) or alfabeto, weights=list(unigr.values()) or None, k=tamanho))
    janela = n - 1
    contexto = escolher_contexto_inicial(modelo, janela, rng)
    saida = []
    for _ in range(tamanho):
        contagens = modelo.get(contexto)
        if not contagens:
            for j in range(1, janela):
                subctx = contexto[j:]
                contagens = modelo.get(subctx)
                if contagens:
                    break
        if not contagens:
            if unigr:
                saida.append(amostrar_com_pesos(unigr, rng))
            else:
                saida.append(rng.choice(alfabeto))
        else:
            prox = amostrar_com_pesos(contagens, rng)
            saida.append(prox)
        contexto = (contexto + saida[-1])[-janela:]
    return "".join(saida)


#### Unigrams

Para esse experimento, iremos olhar apenas quantas vezes cada caracter apareceu no corpus, sem considerar dependência entre caractéres

In [44]:
n=1
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

toi edw ao so tr tcbcem gf  htceryarhthreo ik  itaa iltt e aptrdhe oii guidaln  r twa js o teaoapsyhbedrit nngu s luitt eresbfoluhioug soee nr ua re iptns  ft  ot who d  lrnec  nanedp afh  nai lnhroxh


Nesse ponto, podemos ver que parece que temos ideia de "palavras", onde espaços aparecem, mas de maneira muito "sem sentido" ainda

#### Bigrams

Agora a probabilidade de cada letra é considerada dado a letra anterior, ou seja, quantas vezes aquela letra aparece após a que veio anteriormente

In [45]:
n=2
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

es ofains fo rd thif thes alll t nd buld leximinnd lerunt thind s h t cacr cof sestrohe surer ar folenowalldileyo  but anfffo  awe foflubrercearryong lothedineded ithe tothe t llomar ilend or hecissed


#### Trigrams

In [46]:
n=3
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

ue wheal theas theme ther the ad dideireassmill whishing in bacestel to i a bessel mo th sing by ashindsecom atch   in hescid thumplaysi an loonvader nous of her mand inaund sed st hinsin and norstain


#### 4grams

In [47]:
n=4
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

xlv               the same the her thing toddarty alonger any of my him soots on repompediansweep abasket headylated and unoccasauntenseof leasorbed so comes and the strement a moor crawn withe gents 


#### 5grams

In [48]:
n=5
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

 in the first courteen on the forget so mar           the vittle saw to coministen anxious are the expresent gowanceit nobody ofbarbarabynewalkerbags foot and the first rose stood  north a form i say 


#### 6grams

In [49]:
n=6
modelo, unigr = treinar_ngramas(corpus, n=n)
amostra = amostrar_markov(modelo, unigr, n=n, tamanho=tamanho, alfabeto=alfabeto, seed=seed)
print(amostra)

 interrupted as force our crunchanted words the confident mr meagles away we looking quite eager veins as a warriors of his favor th advanced speech unwearing forwards  it must now you do it about   m


## Conclusão final

Assim como Shannon, parece que à medida que consideramos mais grams o texto vai ganhando “cara” de linguagem, mostrando que de alguma forma, podemos modelar estatísticamente a linguagem. Contudo, temos que confessar que nesse momento, uma modelagem "simples" como essa não nos garante nenhum tipo de entendimento ou "significado".


Ainda na hipótese de Shannon onde a entropia verdadeira da lingua é sempre menor ou igual a de qualquer modelo que busca aproximação, nisso, é como se ainda estivessemos muito "longe" desse limite inferior.


O que nos leva a pergunta, em um modelo hipotético que tem uma entropia igual ou muito próxima da lingua, o quanto podemos dizer que esse modelo é "inteligente". Ou seja, o quanto do mundo real é de fato representado pela língua?. 